# 学習後モデルの性能を調査

## Google Driveとの連携

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ln -s "/content/drive/My Drive/ColabNotebooks" "/content/ColabNotebooks"

## 必要パッケージのインストール

In [ ]:
!pip install -r /content/ColabNotebooks/requirements.txt

## 翻訳 & 評価

### パッケージのimport

In [ ]:
import gensim
import pandas as pd
from gensim.models.word2vec import Word2Vec
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
)
%matplotlib inline
import matplotlib.pyplot as plt

### モデルのロード

In [ ]:
model_path = "/content/ColabNotebooks/model/"
# model_path = "t5-small"

In [ ]:
# トークナイザとモデルの準備
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# 評価モード
model.eval()

In [ ]:
# 検証用モデルの用意
word_model = Word2Vec.load("/content/ColabNotebooks/model_word2vec/word2vec.gensim.model")

### 固定値の決定

In [ ]:
# 生成元のシーケンスの最大長を定義します。
max_length_src = 32

# 生成されるシーケンスの最大長を定義します。
# これは問題の性質によって異なりますが、
# 一般的にはソーステキストの長さの2倍程度を指定することが推奨されます。
max_length_target = 64

# 同じ文の繰り返し（モード崩壊）へのペナルティを定義します。
# 値が大きいほど、生成されるテキストの繰り返しを避けることができます。
# 適切な値は実験によりますが、通常は1.0以上の値を設定します。
repetition_penalty = 8.0

# 生成にランダム性を入れる温度パラメータです。
# 値が小さいほど出力は決定的になり、大きいほど出力はランダムになります。
# 適切な値は問題に依存しますが、一般的には0.7から1.0の間で設定します。
temperature=1.0

# ビームサーチの探索幅を定義します。
# ビームサーチは、生成される各ステップで最良の候補を保持し、
# それらの候補から次のステップを生成します。
# num_beamsが大きいほど、より多くの候補が考慮され、
# 生成されるテキストの質が向上する可能性がありますが、
# 計算コストも増加します。一般的には2から10の間で設定します。
num_beams=4

# 生成結果の多様性を生み出すためのペナルティパラメータです。
# これは特定のトークンが選択されることに対するペナルティを増加させ、
# 結果として生成されるテキストの多様性を高めます。適切な値は問題と目的に依存します。
diversity_penalty=1.0

# ビームサーチのグループ数を定義します。
# これは、ビームを複数のグループに分割し、
# それぞれのグループで独立にサーチを行うことを可能にします。
# これにより、出力の多様性が増加します。
num_beam_groups=4


# 生成する文の数を定義します。
# このパラメータは、複数の異なる出力を生成したい場合に使用します。
# この値はnum_beams以上である必要があります。
num_return_sequences=1

prefix = "translate English to Japanese: "
# prefix = "translate English to French: "

### 生成メソッドの作成

In [ ]:
def generate(
    model: T5ForConditionalGeneration, tokenizer: T5Tokenizer, input: str
) -> str:
    """与えられた入力に対応した、モデルの出力を返します。

    引数:
        model (T5ForConditionalGeneration): 翻訳に使用するモデル

        tokenizer (T5Tokenizer): テキストをトークン化するためのトークナイザ

        input (str): 入力

    戻り値:
        str: 翻訳された日本語の単語
    """
    batch = tokenizer(
        f"{prefix}{input}",
        max_length=max_length_src,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # 生成処理を行う
    outputs = model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        early_stopping=True,
        max_length=max_length_target,
        repetition_penalty=repetition_penalty,
        temperature=temperature,
        num_beams=num_beams,
        diversity_penalty=diversity_penalty,
        num_beam_groups=num_beam_groups,
        num_return_sequences=num_return_sequences,
    )

    generated_texts = [
        tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs
    ]
    return generated_texts[0]

### 翻訳の実行

In [ ]:
input = "Color"
generated_translation = generate(model, tokenizer, input)
print(f"入力: {input} / 出力: {generated_translation}")

入力: Color / 出力: Japanische Farbe


### 評価の実行

In [ ]:
similarities = []

# データフレームの取得
df = pd.read_csv('/content/ColabNotebooks/input/en_and_ja_10.csv')

for index, row in df.iterrows():
    english_word = row["English"]
    correct_translation = row["Japanese"]
    generated_translation = generate(model, tokenizer, english_word)
    is_not_found = False
    try:
        similarity = word_model.wv.similarity(
            correct_translation, generated_translation
        )
    except KeyError:
        is_not_found = True
        print(
            f"単語が見つからないです: {correct_translation} または {generated_translation} がモデルに存在しません。"
        )
    # Word2Vecモデルの学習データに含まれていなかった場合は追加しない
    if not is_not_found:
        similarities.append(similarity)

if len(similarities) > 0:
  plt.plot(similarities)
  plt.show()
  print(f"平均類似度: {sum(similarities) / len(similarities)}")
else: 
  print("データが空です。")